In [77]:
import re
import os
from slugify import slugify
from collections import namedtuple
import markdown
from lxml import etree

In [3]:
text_dir = './tmp/textos-perfil/especies'

# 1. Convertendo para HTML

In [58]:
with open(f"{text_dir}/teste.md", 'r') as f:
    txt = f.read()
    htmlDoc = markdown.markdown(txt,extensions=['extra'])
    
print(htmlDoc)

<p>Aqui vai a descrição geral da espécie. Deixar o texto simples para o leitor leigo. O texto inicial para aparecer no perfil. Usar a tag <glossario>algum termo técnico</glossario> para transformar em tooltip (texto que aparece ao passar o mouse sobre). As seções que devem ser destacadas são indicadas por quatro símbolos (“#”) juntos, separada do título por um espaço, como mostrado abaixo.
O formato destes textos segue o padrão de estrutura do markdown, que pode ser conferido aqui <a href="https://www.markdownguide.org/cheat-sheet/">sobre o markdown</a>. Perceba que o link foi inserido usando este formato <a href="http://link-em-si.com.br">nome do link</a>.</p>
<h1>Onde encontrar</h1>
<p>Aqui entram informações para o usuário sobre como encontrar o animal. Onde procurar, que hora do dia, onde ele costuma cantar, etc.</p>
<h1>Diagnose</h1>
<p>Aqui entram informações sobre como o usuário pode identificar o animal. Quais as características que diferenciam a espécie? Como diferenciar esta 

In [392]:
# Este código pega o html resultante do parse markdown e coloca os elementos em divs correspondentes
soup = BeautifulSoup(htmlDoc)

elements = [i for i in soup.body.children ]
headersIdxs = [i for i,e in enumerate(soup.body.children) if e.name=='h1' ]

elementGroups = [ elements[iStart:iEnd] for iStart,iEnd in zip([0]+headersIdxs, headersIdxs+[None]) ]
elementGroups = [ [ i for i in g if i!='\n' ] for g in elementGroups ]

for i,g in enumerate(elementGroups):
    
    div = soup.new_tag("div")
    div.extend(g)
    div['class'] = slugify( g[0].text if g[0].name=='h1' else 'descricao-geral' )
    soup.html.insert(i,div)


soup.body.decompose()
resultString = "".join(str(i) for i in list(soup.html.children) )
resultString

'<div class="descricao-geral"><p>Aqui vai a descrição geral da espécie. Deixar o texto simples para o leitor leigo. O texto inicial para aparecer no perfil. Usar a tag <glossario>algum termo técnico</glossario> para transformar em tooltip (texto que aparece ao passar o mouse sobre). As seções que devem ser destacadas são indicadas por quatro símbolos (“#”) juntos, separada do título por um espaço, como mostrado abaixo.\nO formato destes textos segue o padrão de estrutura do markdown, que pode ser conferido aqui <a href="https://www.markdownguide.org/cheat-sheet/">sobre o markdown</a>. Perceba que o link foi inserido usando este formato <a href="http://link-em-si.com.br">nome do link</a>.</p></div><div class="onde-encontrar"><h1>Onde encontrar</h1><p>Aqui entram informações para o usuário sobre como encontrar o animal. Onde procurar, que hora do dia, onde ele costuma cantar, etc.</p></div><div class="diagnose"><h1>Diagnose</h1><p>Aqui entram informações sobre como o usuário pode identif

In [388]:
with open('f','w') as f:
    f.write( "".join(str(i) for i in list(soup.html.children) ) )

# Species profile text

In [4]:
def getSpeciesProfileStructure(txt):
    main_desc = re.findall('^([^#]*)',txt,flags=re.DOTALL)[0].strip()
    named_sections = [ (n.strip(),d.strip()) for n,d in re.findall('#+([\w\s]{1,30})\n([^#]+)',txt,flags=re.DOTALL)]
    structured_text = namedtuple('ProfileText',['main_desc','named_sections'])
    return structured_text(main_desc,named_sections)
    return namedtuple('ProfileText', ['main_desc','named_sections'],main_desc,named_sections)

def profileText_to_markdown(profile_txt_str):
    
    resText = ''
    resText += profile_txt_str.main_desc
    
    for name,content in profile_txt_str.named_sections:
        if name.lower()=='curiosidades':
            resText += f'''
<div class="col-lg-12">
  <div class="jumbotron">
    <h1 class="display-4">Você sabia?</h1>
    <hr class="my-4">
    <p>{markdown.markdown(content)}</p>
  </div>
</div>
'''
        
        else:
            resText += f'\n#### {name}\n{content}'
            
    return resText

In [25]:
with open(f"{text_dir}/aplastodiscus-lutzorum.md", 'r') as f:
    txt = f.read()
    html = markdown.markdown(txt)
    
print(html)

<p>﻿Uma das características que mais chama a atenção dessa espécie é a coloração dos seus olhos. O terço superior do olho é dourado e o restante tem a coloração cobre vermelho. A espécie recebe esse nome em homenagem aos cientistas brasileiros Adolfo e Bertha Lutz, que foram pioneiros na descoberta e estudo do gênero Aplastodiscus. Com hábitos florestais e onde há buritizais (<em>Mauritia flexuosa</em>), é encontrada, durante a noite, em matas ribeirinhas cantando empoleirada em em folhas e galhos até 5 metros de altura, o que dificulta estudos com a espécie. Somente foram encontrados indivíduos em locais com mais 1000 metros acima do nível do mar. Machos são encontrados vocalizando entre dezembro e março e uma fêmea já foi encontrada portando grandes <glossario>ovócitos</glossario> em meados de dezembro. O girino da espécie ainda não é conhecido, porém possivelmente são depositados em solo de lama úmida e macia, coberta pela mata densa. <em>A. lutzorum</em> apresenta distribuição dent

In [153]:
import glob

In [262]:
for spfilePath in glob.glob('./tmp/especies/*'):
    species_slug = re.findall('([\w-]*).md$',spfilePath)[0]
    
    with open(spfilePath) as f:
        txt = f.read()
        profile_data = getSpeciesProfileStructure(txt)

    with open(f'../especies/{species_slug}.md','w') as f:
        f.write(profileText_to_markdown(profile_data))
    


In [242]:
species_slug = 'bokermannohyla-pseudopseudis'
text_file = f'../especies/{species_slug}.md'

with open(text_file) as f:
    txt = f.read()
    profile_data = getSpeciesProfileStructure(txt)

with open(f'../especies/{species_slug}-1.md','w') as f:
    f.write(profileText_to_markdown(profile_data))